<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Bias nDCG</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Bias
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(item,user,damping):
    all_recs = []
    test_data = []
    
    bias = Bias(items=item, users=user, damping=damping) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Bias', bias, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
                          
    return results["ndcg"].mean()

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    user = params['user']
    item = params['item']
    damping = params['damping']
    metric = model_trainer(item,user,damping)
    print([item,user,damping,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space={'item': hp.choice('item', ['True', 'False']),
       'user': hp.choice('user', ['True', 'False']),
       'damping': hp.uniform('damping', 0, 1)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

['False', 'True', 0.9551667593742456, 0.0668799781431781]                                                              
,                                                                                                                      
['False', 'True', 0.2168104207674083, 0.05569798930870269]                                                             
,                                                                                                                      
['False', 'True', 0.5059102072736691, 0.05871529458581714]                                                             
,                                                                                                                      
['False', 'False', 0.3991626219441947, 0.055258764459339915]                                                           
,                                                                                                                      
['True', 'True', 0.3234142771816163, 0.0

['False', 'True', 0.8747717974087134, 0.06536088909656537]                                                             
,                                                                                                                      
['False', 'True', 0.9975418025556508, 0.06926099788169031]                                                             
,                                                                                                                      
['False', 'True', 0.8342924428238707, 0.06509021083874734]                                                             
,                                                                                                                      
['False', 'True', 0.9945751920597767, 0.06730131191615367]                                                             
,                                                                                                                      
['False', 'True', 0.8989643097636713, 0.

KeyboardInterrupt: 

In [5]:
rray = [
['False', 'True', 0.9551667593742456, 0.0668799781431781]                      #49.07                         
,                                                                                                                      
['False', 'True', 0.2168104207674083, 0.05569798930870269]                                                             
,                                                                                                                      
['False', 'True', 0.5059102072736691, 0.05871529458581714]                                                             
,                                                                                                                      
['False', 'False', 0.3991626219441947, 0.055258764459339915]                                                           
,                                                                                                                      
['True', 'True', 0.3234142771816163, 0.05539756572457794]                                                              
,                                                                                                                      
['False', 'True', 0.13109432083676054, 0.05474630036513981]                                                            
,                                                                                                                      
['False', 'False', 0.4115990955244868, 0.05467875182492993]                                                            
,                                                                                                                      
['True', 'False', 0.11941506194509433, 0.05454706585247326]                                                            
,                                                                                                                      
['False', 'True', 0.1845511605928244, 0.054282531253012016]                                                            
,                                                                                                                      
['False', 'True', 0.033174616032140225, 0.05432755622314996]                                                           
,                                                                                                                      
['False', 'True', 0.36377049378568793, 0.05538852571138009]                                                            
,                                                                                                                      
['True', 'False', 0.00840884727277369, 0.053740285387489574]                                                           
,                                                                                                                      
['True', 'True', 0.5938666742095927, 0.06016461406840939]                                                              
,                                                                                                                      
['True', 'False', 0.61565814077958, 0.06013068235261894]                                                               
,                                                                                                                      
['False', 'False', 0.8403177332416368, 0.0664547511804861]                                                             
,                                                                                                                      
['False', 'True', 0.9463393360863559, 0.0672989368210832]                                                              
,                                                                                                                      
['False', 'False', 0.7586141124564392, 0.06313380660779978]                                                            
,                                                                                                                      
['True', 'True', 0.7125621251086611, 0.06217747333752192]                                                              
,                                                                                                                      
['False', 'False', 0.6638798026723371, 0.06151577541569757]                                                            
,                                                                                                                      
['True', 'True', 0.6784953351290162, 0.06177024574883942]                                                              
,                                                                                                                      
['False', 'True', 0.9859282997853673, 0.06724132895682368]                                                             
,                                                                                                                      
['False', 'True', 0.9734500062977759, 0.06779879788681309]                                                             
,                                                                                                                      
['False', 'True', 0.8563280980141078, 0.06543324781252374]                                                             
,                                                                                                                      
['False', 'True', 0.9109133335450923, 0.06604164090143895]                                                             
,                                                                                                                      
['False', 'True', 0.7995216494286774, 0.0656918121492713]                                                              
,                                                                                                                      
['False', 'True', 0.9112499804241191, 0.06628158699455264]                                                             
,                                                                                                                      
['False', 'True', 0.9861469924587306, 0.06768213814320531]                                                             
,                                                                                                                      
['False', 'True', 0.5324725598328479, 0.05913535747323232]                                                             
,                                                                                                                      
['False', 'True', 0.9840580017939529, 0.0677724916524438]                                                              
,                                                                                                                      
['False', 'True', 0.9956422398577607, 0.066763705317718]                                                               
,                                                                                                                      
['False', 'True', 0.8580062686714551, 0.06616860877665538]                                                             
,                                                                                                                      
['False', 'True', 0.7561019702775317, 0.06339480345287879]                                                             
,                                                                                                                      
['False', 'True', 0.9105821002467744, 0.06638397165327592]                                                             
,                                                                                                                      
['False', 'True', 0.5734122020682132, 0.059231504487014246]                                                            
,                                                                                                                      
['False', 'True', 0.27589840932833765, 0.055876155290184054]                                                           
,                                                                                                                      
['True', 'True', 0.8013322693309078, 0.06525755009540311]                                                              
,                                                                                                                      
['False', 'True', 0.44324282228954937, 0.056002849038653386]                                                           
,                                                                                                                      
['False', 'False', 0.473267789497003, 0.0575781193113936]                                                              
,                                                                                                                      
['True', 'True', 0.9458088664752854, 0.06649007160970699]                                                              
,                                                                                                                      
['False', 'True', 0.9996837262346565, 0.06800851762019686]                                                             
,                                                                                                                      
['False', 'False', 0.27692866865713783, 0.055073022187858214]                                                          
,                                                                                                                      
['False', 'True', 0.6495116091839978, 0.061647202504919445]                                                            
,                                                                                                                      
['True', 'True', 0.09640686186196701, 0.05407448159051761]                                                             
,                                                                                                                      
['False', 'False', 0.5274998598532117, 0.05896468486277862]                                                            
,                                                                                                                      
['False', 'True', 0.733359654444721, 0.06262596317503491]                                                              
,                                                                                                                      
['True', 'True', 0.8051034230635972, 0.06600903203058334]                                                              
,                                                                                                                      
['False', 'False', 0.8787403405156179, 0.06554762716515554]                                                            
,                                                                                                                      
['False', 'True', 0.20015529504261942, 0.05397177308808303]                                                            
,                                                                                                                      
['True', 'True', 0.34568283071833267, 0.056008500453222546]                                                            
,                                                                                                                      
['False', 'False', 0.6933081044886404, 0.06156143443256165]                                                            
,                                                                                                                      
['False', 'True', 0.6176216556722323, 0.060467690943681394]                                                            
,                                                                                                                      
['True', 'True', 0.9508682478529175, 0.06711263010758206]                                                              
,                                                                                                                      
['False', 'False', 0.8138679433639966, 0.06540986726294064]                                                            
,                                                                                                                      
['False', 'True', 0.7650807127461539, 0.06441060737622566]                                                             
,                                                                                                                      
['False', 'True', 0.8859718460681648, 0.0656240077103502]                                                              
,                                                                                                                      
['True', 'True', 0.399957534349463, 0.055406539867921364]                                                              
,                                                                                                                      
['False', 'False', 0.03868250149529229, 0.05412347617773024]                                                           
,                                                                                                                      
['False', 'True', 0.5731644492011849, 0.0588866573114217]                                                              
,                                                                                                                      
['False', 'True', 0.8353013513558151, 0.06552219190375361]                                                             
,                                                                                                                      
['True', 'True', 0.6306027280228956, 0.06059102852985004]                                                              
,                                                                                                                      
['False', 'False', 0.9383387598938275, 0.06683838844206504]                                                            
,                                                                                                                      
['False', 'True', 0.7158694384044081, 0.06281166089439975]                                                             
,                                                                                                                      
['False', 'True', 0.1541086167998702, 0.054430623894038334]                                                            
,                                                                                                                      
['False', 'True', 0.25434157392780865, 0.055158674274382126]                                                           
,                                                                                                                      
['True', 'True', 0.7698737628929323, 0.064930233092348]                                                                
,                                                                                                                      
['False', 'True', 0.9872587153931979, 0.06723636479640349]                                                             
,                                                                                                                      
['False', 'True', 0.9766052211833923, 0.06719517687989097]                                                             
,                                                                                                                      
['False', 'True', 0.9186579894171584, 0.06618089457784596]                                                             
,                                                                                                                      
['False', 'True', 0.8747717974087134, 0.06536088909656537]                                                             
,                                                                                                                      
['False', 'True', 0.9975418025556508, 0.06926099788169031]                                                             
,                                                                                                                      
['False', 'True', 0.8342924428238707, 0.06509021083874734]                                                             
,                                                                                                                      
['False', 'True', 0.9945751920597767, 0.06730131191615367]                                                             
,                                                                                                                      
['False', 'True', 0.8989643097636713, 0.06583765333310983]                                                             
,                                                                                                                      
['False', 'True', 0.9558667813159376, 0.06601627646500743]                                                             
,                                                                                                                      
['False', 'True', 0.7849731834291509, 0.064573336681599]                                                               
,                                                                                                                      
['False', 'True', 0.848222299918005, 0.0652939515395149]                                                               
,                                                                                                                      
['False', 'True', 0.734737695885632, 0.06226388354301066]                                                              
,                                                                                                                      
['False', 'False', 0.9257769205366012, 0.06654048296598211]                                                            
,                                                                                                                      
['False', 'True', 0.9682272083951203, 0.06660871014930592]                                                             
,                                                                                                                      
['False', 'True', 0.47882165725293835, 0.057066527637088305]                                                           
,                                                                                                                      
['True', 'True', 0.5845104022650758, 0.06017858177310897]                                                              
,                                                                                                                      
['False', 'False', 0.666729181140357, 0.060882572053729725]                                                            
,                                                                                                                      
['False', 'True', 0.4337374969151613, 0.05457549008581928]                                                             
,                                                                                                                      
['False', 'True', 0.9972968275700823, 0.06658801808942005]                                                             
,                                                                                                                      
['False', 'True', 0.6924406017491467, 0.06156740789500763]                                                             
,                                                                                                                      
['True', 'True', 0.8738808590889541, 0.06533753351337454]                                                              
,                                                                                                                      
['False', 'False', 0.5479061816540756, 0.05870915165209845]                                                            
,                                                                                                                      
['False', 'True', 0.8198275623745954, 0.064781040422161]                                                               
,                                                                                                                      
['False', 'True', 0.7403720186523226, 0.0629084932312542]
,
['False', 'True', 0.6887836770654779, 0.05995752969049341]                                                             
,                                                                                                                      
['True', 'True', 0.9000835117642667, 0.06583250433774301]                                                              
,                                                                                                                      
['False', 'True', 0.44779043494164694, 0.05657841704091375]                                                            
,                                                                                                                      
['False', 'False', 0.8250584483770884, 0.06502042822754295]                                                            
,                                                                                                                      
['False', 'True', 0.575487926982116, 0.06023479779314272]                                                              
,                                                                                                                      
['True', 'True', 0.9563851807104697, 0.06664978605820213]                                                              
,                                                                                                                      
['False', 'True', 0.5231211039304127, 0.05890267669549318]                                                             
,                                                                                                                      
['False', 'True', 0.2058097688658111, 0.054457898174916634]                                                            
,                                                                                                                      
['False', 'False', 0.6734473520177486, 0.06169510546789897]                                                            
,                                                                                                                      
['True', 'True', 0.6011199091353776, 0.06122264487881282]                                                              
,                                                                                                                      
['False', 'True', 0.7273351019991176, 0.060928761300484226]]

In [6]:
metric = pd.DataFrame(array)
metric.columns = ['items','users','damping','nDCG']
metric.head(5)

,items,users,damping,nDCG
0,False,True,0.955167,0.066880
1,False,True,0.216810,0.055698
2,False,True,0.505910,0.058715
3,False,False,0.399163,0.055259
4,True,True,0.323414,0.055398


In [7]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,items,users,damping,nDCG
69,False,True,0.997542,0.069261
39,False,True,0.999684,0.068009
21,False,True,0.973450,0.067799
28,False,True,0.984058,0.067772
26,False,True,0.986147,0.067682
71,False,True,0.994575,0.067301
15,False,True,0.946339,0.067299
20,False,True,0.985928,0.067241
65,False,True,0.987259,0.067236
66,False,True,0.976605,0.067195


In [8]:
metric.to_csv('metric_ndcg.csv', index=False)